# Imports and setup

In [1]:
import pyvista as pv
import meshio
import numpy as np
import psycopg2
import hdf5storage
import h5py
import pickle
import logging
from sys import getsizeof
from dotenv import dotenv_values
import matplotlib.pyplot as plt



pv.global_theme.transparent_background = True


config = dotenv_values("../.env")
# Filepaths for script
initMeshPath = config["currentDirectory"] +"data/visualizations/tempMeshes/Trial757_step0.vtu"
outputPath = config["currentDirectory"] + "data/visualizations/controlExperiments/"


In [2]:
# Helper functions for working with database
def get_db_connection():
    conn = psycopg2.connect(
        dbname='simDB',
        user='user',
        password='password',
        host='localhost',
        port='5432'
    )
    return conn

def fetch_trial_data_singleTimestep(conn, trial_id, n=1, start_timestep=0, num_timesteps=500):
    try:
        cur = conn.cursor()
        
        # Fetch data for the given trial_id starting from start_timestep
        query = '''
        SELECT timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat
        FROM simulation_data
        WHERE trial_id = %s AND timestep >= %s AND timestep < %s
        ORDER BY timestep ASC;
        '''
        cur.execute(query, (trial_id, start_timestep, start_timestep + 1))
        rows = cur.fetchall()
        
        # Deserialize data
        data = []
        for row in rows:
            timestep, simulation_time, input_data_bin, output_data_bin, state_data_bin, y_ref_bin, x_hat_bin = row
            input_data = np.array(pickle.loads(input_data_bin))
            output_data = np.array(pickle.loads(output_data_bin))
            state_data = np.array(pickle.loads(state_data_bin))
            y_ref = np.array(pickle.loads(y_ref_bin))
            x_hat = np.array(pickle.loads(x_hat_bin))
            data.append((timestep, simulation_time, input_data, output_data,state_data, y_ref, x_hat))
        
        cur.close()
        return data
    except Exception as e:
        print(f"Error fetching data for trial_id {trial_id}: {e}")
        return None
    


In [3]:
# Trial Parameters
trial_ids = [757]
timesteps = 500
trial_id = trial_ids[0]


In [4]:
# Read in a mesh to get structure
initMesh = pv.read(initMeshPath)
nNodes = initMesh.n_points
nElements = initMesh.n_cells
initMeshFaces = initMesh.cells
print(f"Mesh has {nNodes} nodes and {nElements} elements.")
print(initMeshFaces)

Mesh has 81263 nodes and 721784 elements.
[    3     0  1220 ...  2248 76928 63807]


In [5]:
initMeshPoints = initMesh.points.copy()
initMeshCells = initMesh.cells.copy()

In [6]:
initMeshPoints.shape

(81263, 3)

In [7]:
### Get offsets for reference trajectory and vectorized mesh from database
# Load data from database
conn = get_db_connection()
data = fetch_trial_data_singleTimestep(conn, trial_ids[0], 1, start_timestep=0, num_timesteps=1)
conn.close()
timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
# Get the reference trajectory
refOffsets = np.copy(output_data)


In [8]:
state_data.shape

(81263, 3)

In [9]:
test = initMeshPoints - state_data[0:nNodes, 0:3]

Ran the code below to figure out the permutation between the mesh data saved by sofa as .vtu files and the mesh points saved in the database

It turns out the mesh points saved in the database are perfectly ordered in the same way as the points in the .vtu file. So we'll just extract the face's defined in the .vtu file and use that as the faces at every time step.

If theres ever a point where the permutation mapping is in question though. Can uncomment the code below and find the permutation

In [10]:
# # Find permutation that maps initMeshPoints to state_data
# # This assumes that the points are the almost the same, just in different order
# # We use a tolerance to account for numerical differences
# permutation = []
# used_indices = set() # indices used so far in initial mesh .vtu
# for i, point in enumerate(initMeshPoints):
#     # Subtract point from all state_data points
#     diffs = state_data[:, 0:3] - point
#     dists = np.linalg.norm(diffs, axis=1)
#     # Find the index of the closest point in state_data that hasn't been used yet
#     min_index = np.argmin(dists)
#     # Check if this index has been used
#     while min_index in used_indices:
#         print("Collision detected, finding next closest point")
#         dists[min_index] = float('inf')
#         min_index = np.argmin(dists)
#     permutation.append(min_index)
#     used_indices.add(min_index)
#     if i % 1000 == 0:
#         print(f"Processed {i} points")
# permutation = np.array(permutation)
# print(f"Permutation found with length {len(permutation)} and unique indices {len(set(permutation))}")
# print(permutation)


# # Check that permutation is valid
# if len(permutation) != len(set(permutation)):
#     raise ValueError("Permutation is not valid, some indices are used multiple times")
# # Check that permutation maps initMeshPoints to state_data
# permuted_initMeshPoints = initMeshPoints[permutation]
# max_diff = np.max(np.linalg.norm(permuted_initMeshPoints - state_data[:, 0:3], axis=1))
# print(f"Max difference after permutation: {max_diff}")

# Render animated mesh of robot simulations for each time step

In [11]:
# # Stuff from other file for generating surfaces
#         #### Draw Surface for reference trajectory
#         # Number of samples (in depth dir) and number of points along centerline
#         ### Make surface from desired reference trajectory
#         # Load data from database
#         conn = get_db_connection()
#         data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
#         conn.close()
#         timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
        
#         # Check shape of reference trajectory
#         uncenteredRefTraj = y_ref + refOffsets
#         # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
#         x_refs = uncenteredRefTraj[::2]
#         z_refs = uncenteredRefTraj[1::2]
#         referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
#         path = referenceCurve
#         offsetPath = np.copy(path)
#         offsetPath[:, 1] = offsetPath[:, 1] - 122
 
#         nsamples = 3
#         ntraces = 20  
#         # Define the Z spacing of your 2D section
#         z_spacing = 60
#         # Create structured points draping down from the path
#         points = np.repeat(path, nsamples, axis=0)
#         # repeat the Z locations across
#         tp = np.arange(0, z_spacing * nsamples, z_spacing)
#         # Offset the Z locations 
#         tp = path[:, 1][:, None] - tp
#         points[:, 1] = tp.ravel()
#         # Make a StructuredGrid from the structured points
#         grid = pv.StructuredGrid()
#         grid.points = points
#         grid.dimensions = nsamples, ntraces, 1
#         # Add the data array - note the ordering
#         grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")
        
#         ### Plot actual centerline from output_data
#         # Load data from database
#         conn = get_db_connection()
#         data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
#         conn.close()
#         timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
        
#         # Check shape of output data
#         uncenteredOutputTraj = output_data
#         # Turn output trajectory from 40x1 to 20x2 array by taking every other point
#         x_outs = uncenteredOutputTraj[::2]
#         z_outs = uncenteredOutputTraj[1::2]
#         outputCurve = np.column_stack((x_outs, np.zeros(np.size(x_outs)), z_outs))
#         offsetOutputCurve = np.copy(outputCurve)
#         offsetOutputCurve[:, 1] = offsetOutputCurve[:, 1] - 122
        
#         # load mesh from .vtu file in tempMeshPath
#         mesh = pv.read(tempMeshPath + f"Trial{trial_id}_step{i}.vtu")
        
#         # Add meshes to top view window
#         plotter.subplot(0, 0)
#         plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
#         plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
#         plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name='refPath', opacity=opacities, render_points_as_spheres=True, point_size=10)
#         plotter.add_mesh(pv.PolyData(offsetOutputCurve), color='blue', line_width=5, name='outputPath', opacity=opacities, render_points_as_spheres=True, point_size=10)

In [12]:
# Helper function to that returns pyvista mesh given a trial and timestep from the database 
def get_mesh_for_timestep(trial_id, timestep):
    # Fetch data from database
    conn = get_db_connection()
    data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=timestep, num_timesteps=1)
    conn.close()
    if data is None or len(data) == 0:
        raise ValueError(f"No data found for trial_id {trial_id} at timestep {timestep}")
    timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
    
    # Create a new pyvista mesh with the updated points
    newMesh = pv.PolyData()
    newMesh.points = state_data[0:nNodes, 0:3]
    newMesh.faces = initMeshFaces

    # Generate surface meshes for reference and output trajectories
    uncenteredRefTraj = y_ref + refOffsets
    # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
    x_refs = uncenteredRefTraj[::2]
    z_refs = uncenteredRefTraj[1::2]
    referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
    path = referenceCurve
    offsetPath = np.copy(path)
    offsetPath[:, 1] = offsetPath[:, 1] - 122

    nsamples = 3
    ntraces = 20  
    # Define the Z spacing of your 2D section
    z_spacing = 60
    # Create structured points draping down from the path
    points = np.repeat(path, nsamples, axis=0)
    # repeat the Z locations across
    tp = np.arange(0, z_spacing * nsamples, z_spacing)
    # Offset the Z locations 
    tp = path[:, 1][:, None] - tp
    points[:, 1] = tp.ravel()
    # Make a StructuredGrid from the structured points
    grid = pv.StructuredGrid()
    grid.points = points
    grid.dimensions = nsamples, ntraces, 1
    # Add the data array - note the ordering
    grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")

    ### Plot actual centerline from output_data    
    # Check shape of output data
    uncenteredOutputTraj = output_data
    # Turn output trajectory from 40x1 to 20x2 array by taking every other point
    x_outs = uncenteredOutputTraj[::2]
    z_outs = uncenteredOutputTraj[1::2]
    outputCurve = np.column_stack((x_outs, np.zeros(np.size(x_outs)), z_outs))
    offsetOutputCurve = np.copy(outputCurve)
    offsetOutputCurve[:, 1] = offsetOutputCurve[:, 1] - 122

    ref_mesh = pv.PolyData(offsetPath)
    output_mesh = pv.PolyData(offsetOutputCurve)


    return newMesh, ref_mesh, output_mesh, grid


In [13]:
# Render a single frame to check everything looks good

# mesh, ref_mesh, output_mesh, grid = get_mesh_for_timestep(trial_id, 0)
# # Plot the mesh


# plotter = pv.Plotter(shape=(2,1), border=False, row_weights = [0.4,1], window_size=[1000, 1000])

# plotter.subplot(0, 0)
# plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
# plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
# plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
# plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
# plotter.camera_position = 'yz'
# plotter.camera.roll = 180
# plotter.camera.azimuth = 90
# plotter.camera.elevation = 90
# plotter.enable_parallel_projection()
# # change bounds to be 0, 0, -200, 200, -200, 200
# plotter.camera_position = [
#     (-558.609577998519, -2226.5518997921977, -13.31930130655202),
#     (-558.609577998519, -19.333351135253906, -13.31930130655202),
#     (0.0, -1.0, -2.220446049250313e-16)]
# plotter.camera.zoom(3)

# # Add meshes to isometric view window
# plotter.subplot(1, 0)
# plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
# plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
# plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
# plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
# plotter.camera_position = 'yz'
# plotter.camera.roll = 180
# plotter.camera.azimuth = 40
# plotter.camera.elevation = 50
# plotter.enable_parallel_projection()
# plotter.camera_position = [
#     (528.2333916563371, -1710.160855083042, 898.6502336784569),
#     (-558.609577998519, -19.333351135253906, -13.31930130655202),
#     (0.0, -1.0, -2.220446049250313e-16)]
# plotter.camera.zoom(1.25)
# plotter.show()  




In [ ]:
# Helper function that renders video 
def render_video(trial_id, timesteps):
    # create plotter objects
    plotter = pv.Plotter(shape=(2,1), border=False, row_weights = [0.4,1], window_size=[1008, 1008])
    # Open a mp4
    plotter.open_movie(outputPath + f"Trial{trial_id}_combinedView.mp4", framerate=100, quality=7)
    print(f"Rendering video for Trial {trial_id}...")
    for i in range(timesteps):
        mesh, ref_mesh, output_mesh, grid = get_mesh_for_timestep(trial_id, i)
        # # Plot the mesh
        # if i % 100 == 0:
        #     print(f"Rendering frame {i+1} of {timesteps} for Trial {trial_id}")

        # plotter = pv.Plotter(shape=(2,1), border=False, row_weights = [0.4,1], window_size=[1000, 1000])

        plotter.subplot(0, 0)
        plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
        plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
        plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
        plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 90
        plotter.camera.elevation = 90
        plotter.enable_parallel_projection()
        # change bounds to be 0, 0, -200, 200, -200, 200
        plotter.camera_position = [
            (-558.609577998519, -2226.5518997921977, -13.31930130655202),
            (-558.609577998519, -19.333351135253906, -13.31930130655202),
            (0.0, -1.0, -2.220446049250313e-16)]
        plotter.camera.zoom(3)

        # Add meshes to isometric view window
        plotter.subplot(1, 0)
        plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
        plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
        plotter.add_mesh(ref_mesh, color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
        plotter.add_mesh(output_mesh, color='blue', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=10)
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 40
        plotter.camera.elevation = 50
        plotter.enable_parallel_projection()
        plotter.camera_position = [
            (528.2333916563371, -1710.160855083042, 898.6502336784569),
            (-558.609577998519, -19.333351135253906, -13.31930130655202),
            (0.0, -1.0, -2.220446049250313e-16)]
        plotter.camera.zoom(1.25)
        plotter.write_frame()
    # Close the gif
    plotter.close()





In [ ]:
trials = [201,203,209,216,219,223,230,236]

# [479, 490, 467, 468, 469, 470, 472, 473, 474, 476, 478, 480, 481, 482, 484, 485, 486, 487, 489, 455, 456, 457, 459, 461, 462, 464, 466, 525, 538, 430, 513, 515, 516, 518, 519, 520, 521, 523, 526, 528, 529, 531, 533, 534, 535, 537, 420, 421, 422, 424, 425, 426, 427, 429, 501, 512, 335, 491, 492, 493, 495, 496, 497, 498, 500, 502, 503, 504, 506, 507, 508, 509, 511, 297, 299, 305, 313, 316, 320, 327, 334]

#Redo later [586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 597, 598, 599, 600, 601, 602, 603, 604, 605, 607, 608, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 627, 628, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 646, 647, 648, 649, 650, 651]
# Done - [692, 693, 694, 695, 696, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 679, 680, 682, 683, 685, 686, 687, 688, 689, 690, 691]
# [539, 540, 541, 542, 543, 544, 545, 546, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 653, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671]
timesteps = 500
for trial in trials:
    render_video(trial, timesteps)


Rendering video for Trial 479...


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Rendering video for Trial 490...


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Rendering video for Trial 467...


...

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Rendering video for Trial 327...


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Rendering video for Trial 334...


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [16]:

# plotter = pv.Plotter()
# plotter.open_movie('test.mp4')
# plotter.add_mesh(pv.Sphere())
# plotter.write_frame()
# plotter.close()

